In [ ]:
from jyquickhelper import add_notebook_menu
add_notebook_menu()

# Floyrac Aymeric et Ané Benoit, Projet Python 2A ENSAE ParisTech

<img src="image.jpg", width=800>

# Détection d'anomalies cardiaques à partir d'enregistrements sonores de battements de coeur par phonocardiogramme

Notre projet Python provient du Challenge Physionet 2016 (https://physionet.org/). Ce site propose un challenge tous les ans sur le thème de l'analyse de signaux physiologiques en OpenSource. En 2016, un ensemble d'enregistrements du coeur par phonocardiogramme ont été fournis aux candidats classés dans 6 sous dossiers (a,b,c,d,e et f) afin de construire un modèle prédictif pour déterminer si un coeur bat de façon anormale ou non. Notre objectif était de construire un ensemble de variables explicatives à partir d'une base constituée des signaux bruts afin de bâtir un modèle de Machine Learning. Au préalable, un travail de recherche bibliographique a été necessaire afin de se familiariser avec le vocabulaire du sujet, trouver la méthodologie à appliquer, connaitre les features à extraire ainsi que les modèles ayant déjà été appliqués dans des cas similaires. L'ensemble des articles que nous avons consultés sont renseignés en fin de projet dans la bibliographie.  

Nous pouvons présenter notre travail à travers 5 étapes :  

1) La conversion des enregistrements sonores en matrice exploitable  
2) Notre algorithme qui nous permet de traiter le signal brut pour rendre l'extraction des features plus efficace  
3) la visualisation des données  
4) L'analyse des features  
5) les modèles prédictifs de Machine Learning  
6) Nos commentaires sur le travail effectué  


## Conversion des enregistrements sonores en matrice

L'objectif de cette partie de code est de convertir les enregistrements sonores en matrice exploitable pour notre algorithme. 

In [ ]:
import wave, os, glob
import pandas as pd
import numpy as np
from scipy.io import wavfile

In [ ]:
###On importe chaque .wav pour le transformer en vecteur. Nous bouclons pour chaque fichier .wav se trouvant dans un dossier

zero_a = []
zero_a = pd.DataFrame(zero_a)
path = 'C:\\Users\\benoit\\Desktop\\ENS\\ENSAE\\2A\\S1\\Python\\Projet\\training\\training-a'
for filename in glob.glob(os.path.join(path, '*.wav')):
    fs, data = wavfile.read(filename)
    df = pd.DataFrame(data)
    df = df.transpose()
    zero_a = zero_a.append(df)
    del data, fs

zero_b = []
zero_b = pd.DataFrame(zero_b)
path = 'C:\\Users\\benoit\\Desktop\\ENS\\ENSAE\\2A\\S1\\Python\\Projet\\training\\training-b'
for filename in glob.glob(os.path.join(path, '*.wav')):
    fs, data = wavfile.read(filename)
    df = pd.DataFrame(data)
    df = df.transpose()
    zero_b = zero_b.append(df)
    del data, fs

zero_c = []
zero_c = pd.DataFrame(zero_c)
path = 'C:\\Users\\benoit\\Desktop\\ENS\\ENSAE\\2A\\S1\\Python\\Projet\\training\\training-c'
for filename in glob.glob(os.path.join(path, '*.wav')):
    fs, data = wavfile.read(filename)
    df = pd.DataFrame(data)
    df = df.transpose()
    zero_c = zero_c.append(df)
    del data, fs

zero_d = []
zero_d = pd.DataFrame(zero_d)
path = 'C:\\Users\\benoit\\Desktop\\ENS\\ENSAE\\2A\\S1\\Python\\Projet\\training\\training-d'
for filename in glob.glob(os.path.join(path, '*.wav')):
    fs, data = wavfile.read(filename)
    df = pd.DataFrame(data)
    df = df.transpose()
    zero_d = zero_d.append(df)
    del data, fs
    
zero_e = []
zero_e = pd.DataFrame(zero_e)
path = 'C:\\Users\\benoit\\Desktop\\ENS\\ENSAE\\2A\\S1\\Python\\Projet\\training\\training-e'
for filename in glob.glob(os.path.join(path, '*.wav')):
    fs, data = wavfile.read(filename)
    df = pd.DataFrame(data)
    df = df.transpose()
    zero_e = zero_e.append(df)
    del data, fs

zero_f = []
zero_f = pd.DataFrame(zero_f)
path = 'C:\\Users\\benoit\\Desktop\\ENS\\ENSAE\\2A\\S1\\Python\\Projet\\training\\training-f'
for filename in glob.glob(os.path.join(path, '*.wav')):
    fs, data = wavfile.read(filename)
    df = pd.DataFrame(data)
    df = df.transpose()
    zero_f = zero_f.append(df)
    del data, fs
    
train = []
train = pd.DataFrame(train)
train = zero_a.append(zero_b, ignore_index=True)
train = train.append(zero_e, ignore_index=True)

test = []
test = pd.DataFrame(test)
test = zero_c.append(zero_d, ignore_index=True)
test = test.append(zero_f, ignore_index=True)

#Nous exportons les bases au format .npy
np.save('training_ab.npy', train, allow_pickle=True)
np.save('test_cdf.npy', test, allow_pickle=True)

## Importation de la base
Nous utilisons ici les matrices au format .npy afin d'éviter d'importer des csv et de travailler avec des dataframes. Cette méthode s'est avérée utile dans la mesure où nos ordinateurs travaillaient plus vite avec ces formats

In [ ]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import time
import seaborn as sns
import wave, os, glob
import pandas as pd
import numpy as np
from scipy.io import wavfile
from scipy.signal import find_peaks, welch
from scipy.stats import kurtosis, skew
from scipy.fftpack import fft,fftfreq

In [ ]:
train = np.load('C:\\Users\\benoit\\Desktop\\ENS\\ENSAE\\2A\\S1\\Python\\Projet\\temp\\training_ab.npy')

In [ ]:
train.shape

In [ ]:
test = np.load('C:\\Users\\benoit\\Desktop\\ENS\\ENSAE\\2A\\S1\\Python\\Projet\\temp\\test_cdf.npy')

In [ ]:
test.shape

Nous importons la variables à prédire (car d'un coeur sain ou non) pour chacun des training sets disponibles que nous convertissons en vecteur

In [ ]:
#training : a b d f
labelsa = pd.read_csv('C:\\Users\\benoit\\Desktop\\ENS\\ENSAE\\2A\\S1\\Python\\Projet\\training\\training-a/REFERENCE.csv',header=None,names=['filename','label'])['label']
labelsb = pd.read_csv('C:\\Users\\benoit\\Desktop\\ENS\\ENSAE\\2A\\S1\\Python\\Projet\\training\\training-b/REFERENCE.csv',header=None,names=['filename','label'])['label']
labelsc = pd.read_csv('C:\\Users\\benoit\\Desktop\\ENS\\ENSAE\\2A\\S1\\Python\\Projet\\training\\training-c/REFERENCE.csv',header=None,names=['filename','label'])['label']
labelsd = pd.read_csv('C:\\Users\\benoit\\Desktop\\ENS\\ENSAE\\2A\\S1\\Python\\Projet\\training\\training-d/REFERENCE.csv',header=None,names=['filename','label'])['label']
labelse = pd.read_csv('C:\\Users\\benoit\\Desktop\\ENS\\ENSAE\\2A\\S1\\Python\\Projet\\training\\training-e/REFERENCE.csv',header=None,names=['filename','label'])['label']
labelsf = pd.read_csv('C:\\Users\\benoit\\Desktop\\ENS\\ENSAE\\2A\\S1\\Python\\Projet\\training\\training-f/REFERENCE.csv',header=None,names=['filename','label'])['label']

labels_training = []
labels_training = pd.DataFrame(labels_training)
labels_training = labelsa.append(labelsb, ignore_index=True)
labels_training = labels_training.append(labelse, ignore_index=True)
labels_training = np.asarray(labels_training)
labels_test = []
labels_test = pd.DataFrame(labels_test)
labels_test = labelsc.append(labelsd, ignore_index=True)
labels_test = labels_test.append(labelsf, ignore_index=True)

In [ ]:
labels_training = np.asarray(labels_training)
labels_test = np.asarray(labels_test)

In [ ]:
print(labels_training.shape)
print(labels_test.shape)

## Traitement

Nous pouvons à présent procéder à la construction de notre algorithme. Nous avons utilisé une classe faisant appel à plusieurs fonctions nous permettant d'aboutir à un vecteur de features que nous pourons par la suite exploiter.

Nous avons appris que les battements d'un coeur se décomposent en 4 phases et dans l'odre suivant : S1, systole, S2 et diastole.

In [ ]:
#Nous créons la liste permettant de
phases_list = ['S1','S2','systole','diastole']

L'image suivante présente les différentes phases du battement du coeur

<img src="battement.png", width=800>

## Plan de la class PCG
Nous allons vus présenter, en plus des remarques incorporer dans le code, l'ensemble des fonctions que nous avons crée :  

1) init : Normalisation du signal considéré  
2) envelogram : Calcul de l'envelogramme du signal auquel on applique un seuil pour débruiter le signal. Nous cherchons à calculer l'envelogramme à partir. La détermination de cette nouvelle courbe se fait à partir de l'énergie de Shannon. La formule est également disponible ci-dessous.  

Le graphique suivant nous permet d'illustrer l'envelogramme ce que nous cherchons à calculer

<img src="E.png", width=800>

<img src="enve.png", width=800>

3) consec_count : On compte le nombre de fois qu'il y a une répétition d'une même valeur. Cette fonction sera déterminante pour segmenter les phases cardiaques  
4) segmentation : On segmente le signal en différente phase. Chacune ayant leurs caractéristiques
5) timelengths : On détermine la durée de chaque phase  
6) timefeatures : On extrait les features de durée des phases  
7) computeborne : On détermine les bornes de chaque phase dans le vecteur total afin de pouvoir calculer des valeurs sur chaque phase du signal  
8) moments : On détermine pour chaque phase le kurtosis et l'asymétrie du signal  
9) computeMFCC : "Mel-frequency cepstral coefficients" sont dérivés de l'analyse spectral d'un signal audio  
10) extractFeatures : On extrait l'ensemble de nos features calculés à partir des fonctions précédentes dans un vecteur par signal  

Des notes sont présentes dans le code pour accompagner le lecteur dans sa compréhension du code

In [ ]:
class PCG:
    
    def __init__(self,signal):
        self.signal = np.asarray(signal[~np.isnan(signal)])-np.mean(signal[~np.isnan(signal)]) #on ignore les NaN et on recentre le signal
        self.normalized_signal = self.signal/np.std(self.signal)
        self.fs = 2000
        
    def envelogram(self):
        E = self.signal
        #on remplace les 0 par des petites valeurs (on doit éviter d'avoir un 0 pour calculer les log)
        for i in range(self.signal.shape[0]):
            if E[i]==0:
                E[i] = E[i-1]/10
        E = E/np.max(np.abs(E))
        #E = E/np.std(E)
        E = -1*(E**2) * np.log(E**2)

        #on met les petites valeurs à 0 
        # il faut d'abord calculer les maximums locaux 
        window_length = 750
        local_max = np.array([np.max(E[i-window_length:i+window_length]) for i in range(window_length,len(E)-window_length)])
        local_max = np.concatenate((local_max[0]*np.ones(window_length),local_max,local_max[-1]*np.ones(window_length)))
        s = np.std(E)
        m = np.mean(E)
        mask = E>local_max/5
        mask2 = np.abs(E-m)>s/2
        mask3 = E>np.mean(E)*2
        return E*mask
    
    
    def consec_count(self):
        e = self.envelogram()
        start = 0
        count_mask = []
        while  start < len(e): 
            i = start
            current = e[start]
            while e[i]==current and i < len(e)-1:
                i=i+1
            l = int(i - start + 1)
            count_mask = np.concatenate((count_mask,l*[l]))
            start = i + 1
        return count_mask
    
    def segmentation(self):
        E = self.envelogram()
        count_vec = self.consec_count() # le vecteur de comptage associé
        length_threshold = 50
        beat_mask = count_vec<=length_threshold
        systole_mask = count_vec>length_threshold
        segmentation = np.asarray([])
        mean_beat_length = 0
        beat_counter = 0
        counters = {'S1':0,'S2':0}
        i = 0
        while i<count_vec.shape[0]:
            stop = i 
            next_stop = i
            while stop < count_vec.shape[0] and next_stop-stop<0.050*self.fs:
                stop = next_stop
                #on calcule la longueur du pic
                while stop<count_vec.shape[0] and beat_mask[stop]:
                    stop+=1

                #on calcule l'écart avec le pic suivant, pour voir s'il s'agit du même pic
                next_stop = stop
                while next_stop<count_vec.shape[0] and not beat_mask[next_stop]:
                    next_stop +=1
                
            #un battement long est un battement S1
            if stop-i>mean_beat_length:
                segmentation = np.concatenate((segmentation,np.repeat('S1',stop-i)))
                counters['S1'] += 1
                current_seq = {'beat':'S1','passive':'systole'}
                
            #un battement court est un battement S2
            else:
                segmentation = np.concatenate((segmentation,np.repeat('S2',stop-i)))
                counters['S2'] += 1
                current_seq = {'beat':'S2','passive':'diastole'}
                
            #update beat length and parser
            mean_beat_length = np.average([stop-i,mean_beat_length],weights=[1,beat_counter])
            beat_counter += 1
            i = stop
                
            while stop<count_vec.shape[0] and systole_mask[stop]:
                stop+=1
                
            segmentation = np.concatenate((segmentation,np.repeat(current_seq['passive'],stop-i)))
            #update parser
            i = stop
        
        #on supprime tout ce qui est avant la première phase S1: 
        j = 0 
        while segmentation[j]!='S1':
            j += 1
            
        return segmentation[j:],E[j:],j,counters
    
    
    def timeLengths(self,segmentation,phase):
        res=[]
        start=0
        while start<segmentation.shape[0]:
            i = start
            while i<segmentation.shape[0] and segmentation[i]==phase:
                i+=1
            res.append(i - start)
            start = i + 1
        res = np.array(res)[res!=0]
        if res.shape[0] == 0:
            return 0,0,0
        return np.mean(res),np.std(res),res.shape[0]
    
    def timeFeatures(self,segmentation):
        sysRR = [] #rapport durée systole/durée battement
        diaRR = [] #rapport durée diastole/durée battement
        sysdia = [] # rapport durée systole/durée diastole
        start = 0
        bound = len(segmentation)
        #on commence sur la première phase S1
        while segmentation[start]!='S1':
            start+=1
        #on parcourt toute la segmentation
        while start<bound:
            parser = start
            # on attend d'arriver à la première systole
            while parser<bound and segmentation[parser]!='systole':
                parser += 1
            psys = parser
            # on mesure la taille de la systole
            while psys<bound and segmentation[psys]=='systole':
                psys += 1
            systole_length = psys - parser
            parser = psys
            #on mesure la (les) diastole(s)
            diastole_length_list=[]
            while parser<bound and segmentation[parser]!='S1': #la fin du cycle est marquée par le début d'un nouveau S1
                while parser<bound and segmentation[parser]=='S2':#un S2 précède toujours une diastole
                    parser+=1
                pd = parser
                while pd<bound and segmentation[pd]=='diastole': #on mesure la diastole
                    pd += 1
                diastole_length_list.append(pd-parser)
                parser = pd
                
            #grandeurs extraites
            beat_length = parser - start
            sysRR.append(systole_length/beat_length)
            diaRR.append(sum(diastole_length_list)/beat_length)
            if sum(diastole_length_list)!=0:
                sysdia.append(systole_length/sum(diastole_length_list))
            #update start
            start = parser
        sysRR = np.array(sysRR)
        diaRR = np.array(diaRR)
        sysdia = np.array(sysdia)
        
        return np.nan_to_num([np.mean(sysRR),np.std(sysRR),np.mean(diaRR),np.std(diaRR),np.mean(sysdia),np.std(sysdia)])
   
    def computeBorne(self,segmentation):
        borne = []
        
        for i in range(len(segmentation)-1):
            if segmentation[i-1] != segmentation[i]:
                borne.append(i)
        return borne 
    
    def moments(self,segmentation,borne,phase,decalage):
        phase_skew = []
        phase_kurt = []
        sig = self.signal[decalage:]
        for i in range(0,len(borne)-1):
            if segmentation[borne[i]] == phase:
                interest_signal = sig[borne[i]:borne[i+1]]
                phase_skew.append(skew(interest_signal))
                phase_kurt.append(kurtosis(interest_signal))
        if len(phase_skew) == 0:#cas d'erreur
            return 0,0,0,0
        return np.array([np.mean(phase_skew),np.std(phase_skew),np.mean(phase_kurt),np.std(phase_kurt)])
    
    def computeMFCC(self,borne,phase,decalage):
        mfcc_matrix = np.zeros(10)
        sig = self.signal[decalage:]
        for i in range(0,len(borne)-1):
            if segmentation[borne[i]] == phase:
                interest_signal = sig[borne[i]:borne[i+1]]
                mfccs = mfcc(interest_signal,sr=2000,n_mfcc=10)
                mfcc_matrix = np.row_stack((mfcc_matrix,np.mean(mfccs,axis=0)))
        if len(mfcc_matrix) == 0:#cas d'erreur
            return np.zeros(10)
        
        return  np.mean(mfcc_matrix[1:],axis=1)
        
    def extractFeatures(self):
        segmentation, E,decalage,counters = self.segmentation()
        #ratio nombre de S1 par rapport au nombre de S2
        if counters['S2']!=0:
            features = np.array([counters['S1']/counters['S2']])
        else: #cas d'erreur
            features = np.zeros(1)
        
        #time features
        to_compute_ratio = []
        for phase in phases_list:
            mean,std,last = self.timeLengths(segmentation,phase)
            if phase == 'systole' or phase == 'diastole': #on aura en premier la systole 
                to_compute_ratio.append(last)
            features = np.append(features,[mean,std])
        features = np.append(features,np.nan_to_num(to_compute_ratio[0]/to_compute_ratio[1])) #durée de la systole sur la diastole
        mean_sysRR, std_sysRR,mean_diaRR, std_diaRR,mean_sysdia, std_sysdia = self.timeFeatures(segmentation)
        features = np.concatenate((features,[mean_sysRR, std_sysRR,mean_diaRR, std_diaRR,mean_sysdia, std_sysdia]))
        
        #calcul des bornes    
        p_S1 = []
        p_S2 = []
        borne = self.computeBorne(segmentation)
        
        #Skewness et Kurtosis
        for phase in phases_list:
            features = np.concatenate((features,self.moments(segmentation,borne,phase,decalage)))
        
        #mean12 feature
        features = np.append(features,max(features[2],features[5]))
        
        #Rapport d'amplitudes
        sig = self.signal[decalage:]
        amp_sys_s1 = []
        amp_dia_s2 = []
        
        for i in range(0,len(borne)-2):
            if (segmentation[borne[i]] == 'S1') & (segmentation[borne[i+1]] == 'systole') & (borne[i+2] != 0):
                amp_sys_s1.append(max(sig[borne[i+1]:borne[i+2]]) - min(sig[borne[i+1]:borne[i+2]])/max(sig[borne[i]:borne[i+1]]) - min(sig[borne[i]:borne[i+1]]))
            else :
                pass
        for i in range(0,len(borne)-2):
            if (segmentation[borne[i]] == 'S2') & (segmentation[borne[i+1]] == 'diastole') & (borne[i+2] != 0):
                amp_dia_s2.append(max(sig[borne[i+1]:borne[i+2]]) - min(sig[borne[i+1]:borne[i+2]])/max(sig[borne[i]:borne[i+1]]) - min(sig[borne[i]:borne[i+1]]) )
            else :
                pass
        if len(amp_sys_s1)==0:
            features = np.append(features,[0,0])
        else: 
            features = np.append(features,[np.mean(amp_sys_s1),np.std(amp_sys_s1)])
        if len(amp_dia_s2)==0:
            features = np.append(features,[0,0])
        else:
            features = np.append(features,[np.mean(amp_dia_s2),np.std(amp_dia_s2)])

        #spectral features
        frequency_bands = np.array([0,25,40,60,90,120,160,250,400])
        freqs, psd = welch(self.signal[decalage:],fs=2000)
        freqs, psd = np.asarray(freqs),np.asarray(psd)
        band_spectral_power = []
        for i in range(frequency_bands.shape[0]-1):
            mask = [freqs<frequency_bands[i+1]] and [freqs>frequency_bands[i]]
            band_spectral_power.append(np.sum(psd[mask]))
        features = np.append(features,band_spectral_power)
        
        return features

# Visualisation

Cette partie va nous permettre de nous familiariser avec nos features à travers des représentations graphiques. Nous proposons une illustration de notre segmentation pour un signal type ainsi que des plots présentant les features que nous venons de construire

In [ ]:
time_length = 10000
sample = train[87][:time_length]

In [ ]:
pcg = PCG(sample)
segmentation,E,lag,c = pcg.segmentation()

In [ ]:
plt.figure(figsize=(14,5))
l = time_length-lag
x = np.linspace(0,l/2000,l)
plt.fill_between(x,0,1,where = segmentation=='S1',facecolor='r',alpha=.3,label='S1')
plt.fill_between(x,0,1,where = segmentation=='S2',facecolor='r',alpha=.1,label='S2')
plt.fill_between(x,0,1,where = segmentation=='systole',facecolor='b',alpha=.3,label='systole')
plt.fill_between(x,0,1,where = segmentation=='diastole',facecolor='b',alpha=.1,label='diastole')

plt.plot(x,E/np.max(E),'k',label='signal')
plt.legend()
plt.xlabel('temps (s)')
plt.ylabel('amplitude normalisée')
plt.show()

## Test MFCC

In [ ]:
features = np.zeros(20)
for i in range(train.shape[0]):
    sample = train[i]
    pcg = PCG(sample)
    segmentation,
    mfccs = mfcc(sample,n_mfcc=20,sr=2000)
    features = np.row_stack((features,np.min(mfccs,axis=1)))
features = features[1:]

In [ ]:
fig = plt.figure(figsize=(14,20))
for i in range(20):
    ax = fig.add_subplot(10,2,i+1)
    tho = features[:,i]
    ax.hist(tho[labels==1],density=True,alpha=0.5,color='b')
    ax.hist(tho[labels==-1],density=True,alpha=0.5,color='r')

# Analyse des features
L'analyse de feature va nous permettre de déterminer le lien entre les variables que nous avons réussi à dégager et l'anomalie cardiaque. Nous proposerons également une comparaison graphique des variables entre des coeurs qui battent de façon normal et de façon anormale.

In [ ]:
start = time.time()
sample = train[408]
pcg = PCG(sample)
ft = pcg.extractFeatures()
end = time.time()
print(end-start)
ft.shape

In [ ]:
n_samples = train.shape[0]

In [ ]:
print(n_samples)

In [ ]:
X = np.zeros(45)
final_labels = []
for i in range(n_samples):
    try:
        sample = train[i]
        pcg = PCG(sample)
        ft = pcg.extractFeatures()
        X = np.row_stack((X,ft))
        final_labels.append(labels[i])
        
    except:
        print('il y a une erreur avec le sample %d'%i)
X = X[1:]

In [ ]:
np.save('features.npy',X)

In [ ]:
fig = plt.figure(figsize=(14,35))
X = np.asarray(X)
final_labels = np.array(final_labels)
for i in range(51):
    ax = fig.add_subplot(17,3,i+1)
    ft = X[:,i]
    #print(ft)
    normalft = ft[final_labels==1]
    anormalft = ft[final_labels==-1]
    ax.hist(normalft,density=True,color='b',alpha=0.5)
    ax.hist(anormalft,density=True,color='r',alpha=0.5)

In [ ]:
#Convertion en Df au cas où
X = np.column_stack((t1,t2,t12,t21,m12,ps1,ps2,good))
X_df = pd.DataFrame(X, columns=["t1","t2","t12","t21","mean12","S1","S2","Normal"])
X_df["Normal"] = X_df["Normal"].map({0 : "anormal", 1 : "normal"})

In [ ]:
#Matrice de corrélation
sns.heatmap(X_df.corr())

In [ ]:
#Violin plot per variable
features_list = X_df.columns[:-1]
for i in features_list:
    fig = plt.figure()
    fig = plt.figure(figsize=(5,5))
    ax = sns.violinplot(x = [i]*len(X_df), y=X_df[i],hue=X_df["Normal"], palette="Set2", split = True)

# Classification
Dans cette partie, nous allons appliquer des algorithmes de Machine Learning à nos features afin de détecter si un coeur est malade ou non
Dans la mesure où notre problème est un cas de classification, nous avons considéré plusieurs modèles. Nos modèles benchmark sont le modèle qui prédit qu'un coeur est toujours normal ainsi que le modèle Logit. Par la suite, nous partirons d'un Random Forest et d'un SVM afin d'élaborer un modèle prédictif optimal.

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import normalize

In [ ]:
pca = PCA(n_components = 2)
X_red = pca.fit_transform(X)

In [ ]:
rf = RandomForestClassifier(n_estimators=10,max_depth=2)
svm = SVC(C=100,gamma=100)
nn = MLPClassifier()
logit = LogisticRegression()
xtrain,xtest,ytrain,ytest = train_test_split(X,final_labels,test_size = 0.5)
for clf in [logit,rf,svm]:
    clf.fit(xtrain,ytrain)
    print(confusion_matrix(clf.predict(xtest),ytest)/len(ytest))

In [ ]:
np.sum(final_labels==1)/len(final_labels)

In [ ]:
features = np.column_stack((t1,t2,t12,t21,ps1,ps2,m12))
features.shape

In [ ]:
labels_=y = X_df.loc[:,"Normal"].values
labels_.shape

In [ ]:
#Logit
X_train, X_test, y_train, y_test = train_test_split(features, labels_, test_size=0.20)
logit = LogisticRegression()
result = logit.fit(X_train, y_train)
predictions = logit.predict(X_test)
print(accuracy_score(y_test, predictions))

In [ ]:
#RandomForest
X_train, X_test, y_train, y_test = train_test_split(features, labels_, test_size=0.20)
RF =  RandomForestClassifier(n_estimators=150, max_depth=7, random_state=0)
result = RF.fit(X_train, y_train)
predictions = RF.predict(X_test)
print(accuracy_score(y_test, predictions))

In [ ]:
#SVM
X_train, X_test, y_train, y_test = train_test_split(features, labels_, test_size=0.20)
clf = SVC()
result = clf.fit(X_train, y_train)
predictions = clf.predict(X_test)
print(accuracy_score(y_test, predictions))

In [ ]:
rf = RandomForestClassifier()
svm = SVC()
print(cross_val_score(svm,features,labels_,cv=3))

In [ ]:
#RF
param_grid = {'n_estimators': [100, 150, 200, 250, 300],'max_depth': [2, 4, 6, 7]}
grid_RF = GridSearchCV(RF, param_grid, cv=10)
grid_RF.fit(X_train, y_train)
grid_RF. best_estimator_
grid_RF. best_params_

In [ ]:
#Pour un SVM
tuned_parameters = [{'kernel': ['rbf'], 'gamma': [1e-3, 1e-4, 1e-5, 1e-6],'C': [1, 10, 100, 1000]}]
print("# Tuning hyper-parameters for %s" % 'precision')
print()
clf = GridSearchCV(SVC(), tuned_parameters, cv=5)
clf.fit(X_train, y_train)
print("Best parameters set found on development set:")
print()
print(clf.best_params_)
print()
print("Grid scores on development set:")
print()

# Conclusion
Bof  
Critiques ?  
Ouverture ?

## Bibliographie

Moukadem, A. (2011). Segmentation et classification des signaux non-stationnaires: application au traitement des sons cardiaque et à l'aide au diagnostic (Doctoral dissertation, Université de Haute Alsace-Mulhouse).

Liang, H., Lukkarinen, S., & Hartimo, I. (1997, September). Heart sound segmentation algorithm based on heart sound envelogram. In Computers in Cardiology 1997 (pp. 105-108). IEEE.

Safara, F., Doraisamy, S., Azman, A., Jantan, A., & Ramaiah, A. R. A. (2013). Multi-level basis selection of wavelet packet decomposition tree for heart sound classification. Computers in biology and medicine, 43(10), 1407-1414.

Singh, M., & Cheema, A. (2013). Heart sounds classification using feature extraction of phonocardiography signal. International Journal of Computer Applications, 77(4).

Atbi, A., & Debbal, S. M. (2013). Segmentation of pathological signals phonocardiogram by using the Shannon energy envelogram. AJCM, 2(1), 1-14.